# Explore and visualize the philogeny provided by phagescope

In [6]:
import numpy as np
import pandas as pd
import os
#import dask.dataframe as dd

In [12]:
#Load PHAGE METADATA in a pandas dataframe

# Path to the folder containing TSV files -> PROTEIN METADATA
folder_path = r'C:\Users\lorenzo\Desktop\datasets\phagescope\Metadata\phage_metadata\*.tsv'  # Use a wildcard to select all TSV files
folder_path = folder_path.replace(os.sep, '/')
# Load all TSV files into a single Dask DataFrame
#start = time.time()
phage_metadata = dd.read_csv(folder_path, sep='\t')

#print("elapsed time: " +  str(time.time() - start))

# Optionally, print the first few rows (this won't trigger computation yet)

# For now open only the EMBL dataset

phage_metadata = dd.read_csv(r"C:\Users\lorenzo\Desktop\datasets\phagescope\Metadata\phage_metadata\embl_phage_meta_data.tsv", sep='\t')

phage_metadata.head()



,Phage_ID,Length,GC_content,Taxonomy,Completeness,Host,Lifestyle,Cluster,Subcluster
0,AJ550635.1,4554,41.040843,Microviridae,High-quality,Chlamydophila pecorum,virulent,cluster_152500,subcluster_191945
1,AJ969242.1,7878,23.013455,NaN,Not-determined,Spiroplasma citri,virulent,cluster_24190,subcluster_30305
2,FQ482084.1,45522,49.681473,Caudovirales,High-quality,Erwinia amylovora,virulent,cluster_223352,subcluster_281280
3,FR725450.1,23750,52.985263,Caudovirales,Medium-quality,Nitrososphaera viennensis EN76,virulent,cluster_67564,subcluster_85097
4,AJ536073.2,14931,39.729422,NaN,High-quality,Bacillus thuringiensis serovar israelensis,temperate,cluster_26036,subcluster_32746


In [14]:
phage_metadata = phage_metadata[['Phage_ID', 'Cluster', 'Subcluster']]
phage_metadata.head()

,Phage_ID,Cluster,Subcluster
0,AJ550635.1,cluster_152500,subcluster_191945
1,AJ969242.1,cluster_24190,subcluster_30305
2,FQ482084.1,cluster_223352,subcluster_281280
3,FR725450.1,cluster_67564,subcluster_85097
4,AJ536073.2,cluster_26036,subcluster_32746


In [18]:
from ete3 import Tree
import ete3

df = phage_metadata

# Generate Newick String
clusters = {}
for _, row in df.iterrows():
    cluster = row['Cluster']
    subcluster = row['Subcluster']
    phage = row['Phage_ID']
    if cluster not in clusters:
        clusters[cluster] = {}
    if subcluster not in clusters[cluster]:
        clusters[cluster][subcluster] = []
    clusters[cluster][subcluster].append(phage)

def build_newick(clusters):
    def format_subtree(subtree):
        if isinstance(subtree, dict):
            return "(" + ",".join([f"{format_subtree(v)}" for k, v in subtree.items()]) + ")"
        elif isinstance(subtree, list):
            return ",".join(subtree)
    return "(" + ",".join([f"{format_subtree(v)}{k}" for k, v in clusters.items()]) + ");"

newick = build_newick(clusters)
print("Newick String:", newick)

# #reate ete3 tree form newick string
ete_tree1 = Tree(newick, format=1)
ts = ete3.TreeStyle()
ts.show_branch_length = True
ts.show_branch_support = True
ts.show_leaf_name = True
# ts.mode = "c"
# ts.arc_span= 180
# ts.arc_start= -180

ete_tree1.show(tree_style=ts)


Newick String: ((AJ550635.1,AJ270057.1)cluster_152500,(AJ969242.1)cluster_24190,(FQ482084.1,FQ482086.1)cluster_223352,(FR725450.1)cluster_67564,(AJ536073.2)cluster_26036,(HE983844.1,HF543949.1)cluster_262131,(HG428758.1)cluster_78863,(LN610575.1)cluster_233893,(LN610576.1)cluster_233205,(LN610581.1,LN610582.1,LN610583.1,LN610573.1)cluster_268360,(LN610585.1)cluster_186886,(LN610586.1,HE983845.1)cluster_229453,(LN610587.1)cluster_177538,(LN610588.1,FM201281.1)cluster_11026,(LN681534.1)cluster_113235,(HG796219.1,HG796220.1,HG793132.1)cluster_268361,(HG796221.1)cluster_177539,(LR031359.1)cluster_152445,(X15031.1)cluster_250815,(X60322.1,X60323.1)cluster_158413,(X52107.1)cluster_67555,(X14336.1)cluster_63254,(X02139.1)cluster_215731,(X03869.1)cluster_68527,(V00642.1)cluster_110885,(X07489.1)cluster_39628,(V00657.1)cluster_177517,(Z18946.1)cluster_272329,(X58839.1)cluster_229441,(X51344.1)cluster_64617,(V01146.1)cluster_251005,(X51522.1)cluster_87300,(X80191.1)cluster_33704,(X99260.1)cluste